In [2]:
# https://www.aanda.org/articles/aa/pdf/2006/45/aa5897-06.pdf

In [3]:
import astropy
import math
import numpy as np
from astropy.time import Time
from astropy.coordinates import Angle
import numpy.polynomial.polynomial as poly

In [4]:
def rot1(ang):
    return [[1,0,0], 
            [0, np.cos(ang), np.sin(ang)], 
            [0, -np.sin(ang), np.cos(ang)]
           ]

def rot2(ang):
    return [ 
            [np.cos(ang), 0, -np.sin(ang)],
            [0, 1, 0],
            [np.sin(ang), 0, np.cos(ang)]
           ]

def rot3(ang):
    return [[np.cos(ang), np.sin(ang), 0], 
            [-np.sin(ang), np.cos(ang), 0],
            [0, 0, 1]
           ]

In [8]:
# Earth Rotation Angle
def era(thist):
    tu = thist.ut1.jd - 2451545.0
    a = 0.7790572732640 + 1.00273781191135448 * tu
    a = a % 1
    return Angle(2 * math.pi * a, unit='rad')

In [9]:
thist = Time("2006-01-15T21:24:37.5", scale='utc')
print(thist.tt.mjd)
j2000 = Time("J2000", format='jyear_str', scale="tt")
tc = (thist - j2000).to_value('year') / 100
print(tc) 
print(thist.ut1.mjd)

53750.89285513889
0.06040774415164652
53750.89210456099


In [87]:
era_ang = era(thist)
era_ang.value

1.3310828750217287

In [13]:
def mat_prec1(X, Y, s):
    return [[1, 0, -X.rad], 
            [0, 1, -Y.rad],
            [X.rad, Y.rad, 1]
           ]

def mat_prec2(X, Y, s):
    x = X.rad
    y = Y.rad
    s = 0
    x4 = (1+x**2/4)
    x24 = x4 / 2
    x22 = (1-x**2/2)
    y22 = (1-x**2/2)
    
    return [[1-x**2*x24, -s-x*y*x24, -x-s*y], 
            [s*x22-x*y*x24, y22, -y - s *x],
            [x, y, 1-(x**2*x4-y**2)/2]
           ]

In [15]:
j2000 = Time("J2000", format='jyear_str', scale="tt")

In [30]:
thist.tt.jd

2453751.392855139

In [31]:
import erfa

In [66]:
thist = Time("2006-01-15T21:24:37.5", scale='utc')
utc1, utc2 = erfa.dtf2d('UTC', 2006,1,15,21,24,37.5)
tai1, tai2 = erfa.utctai(utc1, utc2)
tt1, tt2 = erfa.taitt(tai1, tai2)
print(utc1, utc2)
print(tt1, tt2)
thist.to_value(format='jyear')

2453750.5 0.8921006944444444
2453750.5 0.8928551388888888


2006.0407723496082

In [33]:
erfa.s06a(tt1, tt2) #rad

-1.2469339751752158e-08

In [41]:
mcio = erfa.c2i06a(tt1, tt2)
Xx, Yy, Ss = erfa.xys06a(tt1, tt2)

In [75]:
rmat = rot3(era_ang) @ mcio
rmat

array([[ 2.37424217e-01,  9.71406047e-01, -1.79207501e-04],
       [-9.71405888e-01,  2.37424281e-01,  5.58274894e-04],
       [ 5.84859820e-04,  4.15352420e-05,  9.99999828e-01]])

In [76]:
sp = erfa.sp00(tt1, tt2)

In [80]:
# http://hpiers.obspm.fr/eoppc/eop/eopc01/filtered-pole.tab
# Polar motion
# values in arcsec
thist.to_value(format='jyear')
xp = Angle(0.061017, unit='arcsec')         
yp = Angle(0.345285, unit='arcsec')

In [81]:
rpm = erfa.pom00(xp.rad, yp.rad, sp)
rpm

array([[ 1.00000000e+00, -1.37646554e-11,  2.95818764e-07],
       [ 1.42598527e-11,  1.00000000e+00, -1.67398892e-06],
       [-2.95818764e-07,  1.67398892e-06,  1.00000000e+00]])

In [82]:
rglob = rpm @ rot3(era_ang) @ mcio
rglob

array([[ 2.37424217e-01,  9.71406047e-01, -1.78911682e-04],
       [-9.71405889e-01,  2.37424281e-01,  5.56600905e-04],
       [ 5.83163463e-04,  4.16453275e-05,  9.99999829e-01]])

In [83]:
# GMST
erfa.gmst06

<function erfa.core.gmst06(uta, utb, tta, ttb)>

In [85]:
erfa.era00(thist.ut1.jd, 0)

1.3310828750206056

In [88]:
1.3310828750217287

1.3310828750217287

In [89]:
dec = Angle(57, unit='deg')
ra = Angle(120, unit='deg')

In [90]:
v1 = (np.cos(dec)*np.cos(ra), np.cos(dec)*np.sin(ra), np.sin(dec))

In [91]:
v2 = rglob @ v1

In [92]:
v2

array([0.393379  , 0.37698579, 0.83853126])

In [102]:
dec2 = Angle(np.arcsin(v2[2]), unit='rad')
ra2 = Angle(np.arctan2(v2[0], v2[1]), unit='rad')

In [103]:
dec2.deg

56.985347827950605

In [104]:
ra2.deg

46.219058551954696

In [116]:
long = Angle(0.23, unit='deg')
phi = Angle(41, unit='deg')

In [117]:
h = era_ang + long - ra2

In [118]:
h.hourangle

2.0184248245953325

In [119]:
dec2, h

(<Angle 0.99458194 rad>, <Angle 0.52842238 rad>)

In [126]:
sin_a = np.sin(dec2) * np.sin(phi) + np.cos(dec2) * np.cos(phi) * np.cos(h)
a = Angle(np.arcsin(sin_a)).deg

In [127]:
a

64.85631248620432